In [11]:
from openai import OpenAI
import time
import re
import json
import os
from dotenv import load_dotenv
import sys 
sys.path.append('..')
import requests
from src.LLM_prompting import *
from src.html_generation import * 
import pdfkit 
from src.CvBuilder import CvBuilder
from src.MongoController import MongoController

In [2]:

load_dotenv()
model_choice = os.getenv("OPENAI_MODEL_CHOICE")
openai_api_key = os.getenv("OPEN_AI_API_KEY")

client = OpenAI(
    api_key=openai_api_key
)
system = {
    "model_choice" :  model_choice,
    'system_prompt' : "You are an helpful assistant, you will only use provided informations to answer and will not assume informations. You need to output only relevant information, introduction message are not required"
}

In [3]:
mongo_controller = MongoController()
builder = CvBuilder(mongo_controller, client, system)

In [4]:
builder.load()

State 672de255c9ccc52572856f47 loaded for user 672918ed6de6f79da9ef60f5


In [18]:
def generate_cover_letter_html(company_name,position_title,full_name,cover_letter_content):
    template_env = Environment(loader=FileSystemLoader('.'))
    cover_letter_template = template_env.get_template('templates/cover_letter_template.html')


    context = {
        "company_name" : company_name,
        "position_title" : position_title,
        "full_name" : full_name,
        "cover_letter_content" : cover_letter_content
    }

    # Render the template with the context
    cover_letter_html = cover_letter_template.render(context)
    return cover_letter_html


html_content = generate_cover_letter_html(builder.company_info['name'],
                            builder.company_info['position_title'],
                            builder.coordinates['full_name'],
                            builder.cover_letter.replace('\n','<br>'))

In [19]:

try:
    pdfkit.from_string(html_content, './output/cover.pdf')
    print('Cover letter created!')
except Exception as e:
    print(f"Error generating PDF: {e}")

Cover letter created!


In [13]:
builder.generate_cover_letter_pdf('./output_file/cover_letter.pdf')

Error generating PDF: wkhtmltopdf exited with non-zero code 1. error:
QPainter::begin(): Returned false
Exit with code 1, due to unknown error.



In [ ]:
with open('../job_description.txt','r') as fd:
    job_description = fd.read()

builder.generate_cv_from_job_description(job_description)


In [6]:
builder.save()

State saved InsertOneResult(ObjectId('67291ab592bcaf074801f8ce'), acknowledged=True)


In [6]:
builder.generate_cv_pdf('../output/cv.pdf')

CV created!


In [5]:

# user_data['static_context']
# user_data['general_info'] 
# user_data['skills']
# user_data['work_experience']
# user_data['work_experience_meta'] 
# user_data['education'] 

{'static_context': {'full_name': 'Gabriel Ludel', 'location': 'Tallinn', 'email': 'gabriel.ludel@gmail.com', 'phone': '<b>+33767534950</b>', 'portfolio_link': 'https://gludel.com'}, 'general_info': "**Professional Background:**\n1. **Roles and Responsibilities**:\n   - Former Lead Fullstack Developer at my previous company.\n   - Managed the complete IT infrastructure for the company.\n   - Built several production applications, including an ETL, CRM, and user applications from scratch.\n   - Experienced in managing the full life cycle of a web application.\n\n2. **Technical Skills**:\n   - Fullstack development (4 years of experience).\n   - Defined application infrastructure using microservices, DevOps, Docker, CI/CD pipelines, and software design patterns.\n   - Experience in DevOps and data engineering, including data analysis.\n   - Knowledge of domain and hosting environments, including setup and management.\n\n3. **Educational Background**:\n   - Bachelor's degree in Computer Sc

---

From json to mongo

In [6]:
with open('./personal_info.json','r') as fd:
    user_data = json.load(fd)

mongo_controller = MongoController()
mongo_controller.save_user_info(user_data)


InsertOneResult(ObjectId('672918ed6de6f79da9ef60f5'), acknowledged=True)

In [14]:

state = {
    'job_infos': builder.job_infos,
    'work_context': builder.work_context,
    'project_context': builder.project_context,
    'relevant_skillset': builder.relevant_skillset,
    'profile_description': builder.profile_description,
    'profile_title': builder.profile_title,
    'job_description' : job_description
}
user_id = builder.user_id

mongo_controller.save_user_cv(user_id,state)

InsertOneResult(ObjectId('672919086de6f79da9ef60f7'), acknowledged=True)

In [16]:

url = 'http://localhost:80/pdf'
html_content = "<html><head><meta charset='utf8'></head><body>Hello world</body></html>"

def convet_html_to_pdf(html_content,output_path):
    # Send a POST request with the HTML content
    response = requests.post(url, data={'html': html_content})
    print(response)
    # Check if the request was successful
    if response.status_code == 200:
        # Write the response content to a PDF file
        with open(output_path, 'wb') as file:
            file.write(response.content)
    else:
        print(f"Failed to generate PDF. Status code: {response.status_code}")

<Response [200]>
PDF saved successfully as 'youramazingfile.pdf'


In [71]:
data = {
  "coordinates": {
        "full_name": "Gabriel Ludel",
        "location": "Tallinn",
        "email": "gabriel.ludel@gmail.com",
        "phone": "+33767534950",
        "portfolio_link": "https://gludel.com",
        "availability": "now"
    },
    "general_info": [
        {
            "name" : "Roles and Responsibilities",
            "infos" : [
                "Former Lead Fullstack Developer at my previous company.",
                "Managed the complete IT infrastructure for the company.",
                "Built several production applications, including an ETL, CRM, and user applications from scratch.",
                "Experienced in managing the full life cycle of a web application."
            ],
        },
        { 
            "name" : "Technical Skills" ,
            "infos" : [
                "Fullstack development (4 years of experience).",
                "Defined application infrastructure using microservices, DevOps, Docker, CI/CD pipelines, and software design patterns.",
                "Experience in DevOps and data engineering, including data analysis, cloud native applications.",
                "Knowledge of domain and hosting environments, including setup and management."
            ],
        },
        {
            "name" : "Educational Background",
            "infos" : [
                "Bachelor's degree in Computer Science.",
                "Master's degree in Machine Learning."
            ],
        },
        {
            "name" : "Industry Experience",
            "infos" : [
                "Experience working in industrial, corporate, and startup environments.",
            ],
        }
        ,
        {    
            "name" : "Additional Information",
            "infos" : [
                "Passionate about computer science and artificial intelligence.",
                "Own a self-hosted LLM station for AI model experimentation.",
                "Proactive and get the job done ",
                "Used to find complexe solutions in constraint environment"
            ],
        }
    ],       
    "work_experience": {
        "intrad": {
            "company_name" : "Intrad",
            "title": "Fullstack Developer",
            "start_date": "july 2023",
            "end_date": "October 2024",
            "employment_type": "Full-time",
            "responsibilities":[
                    "DevOps: Cloud infrastructure selection, deployment scripts (YAML, Bash), database management, domain and hosting management, Git management.",
                    "E-learning Application Development: Created quiz forms, PDF embedding, certificate generation.",
                    "CRM Development: API creation for remote control, integration with Discord, Zoom, Vimeo, GSuite, user authentication, email management, customer support, monitoring.",
                    "Learning Management System (LMS): User resource access control, video tracking, mobile and desktop compatibility."
            ],
            "skills" : ["Microservices", "Bash", "cloud infrastructure", "APIs", "Zoom API", "GSuite", "UX/UI design", "database management", "python", "django", "aws", "docker", "vue.js", "node.js", "CI/CD"]
        },
        "freelancing": {
            "company_name" : "Freelancing",
            "title": "Fullstack Developer",
            "start_date": "september 2020",
            "end_date": "October 2024",
            "employment_type": "Freelance",
            "responsibilities":[
                    "ML Engineer: Developper ML models for financial forecasting and reinforcement learning taks",
                    "Web Development: Responsive, multi-language websites; Fullstack web applications."
            ],
            "skills" : ["Django", "Vue", "UX/UI", "Sktlearn", "Scikit", "Pytorch", "Tensorflow", "Pandas", "Numpy"]
        },
        "helioit": {
            "company_name" : "HelioIT",
            "title": "Fullstack Developer",
            "start_date": "july 2023",
            "end_date": "Today",
            "employment_type": "Lead dev, Full-time",
            "responsibilities":[
                    "SAAS Development: Backend development, database schema design, Vue.js app development, product ownership.",
                    "ChatBot Application: Integrated LLM APIs, OpenAI wrapper, self-hosting LLM models, multilingual support."
            ],
            "skills" : ["Selfhosting AI","LLM APIs", "OpenAI", "multilingual application design", "Vue.js", "nest.js", "typescript", "API"]
        },
        "actinvision": {
            "company_name" : "ActinVision",
            "title": "Data Analyst",
            "start_date": "september 2022",
            "end_date": "july 2023",
            "employment_type": "Full-time",
            "responsibilities":[
                    "Data Analysis: Created dashboards, used Tableau Online for data visualization, data modeling with Dataiku."
            ],
            "skills" : ["Tableau", "Dataiku", "data modeling"]
        },
        "hager group": {
            "company_name" : "Hager Group",
            "title": "Data Manager",
            "start_date": "january 2022",
            "end_date": "july 2022",
            "employment_type": "Internship",
            "responsibilities":[
                    "ETL Processing: Handled large volumes of Excel data, POC/MVP development, data export into formatted Excel.",
                    "Machine Learning Models: Built forecasting models using Scikit-Learn."
            ],
            "skills" : ["Excel", "data processing", "Scikit-Learn"]
        }
    },

    "skills": {
        "front" : ["html", "css", "vue.js", "react", "ux/ui", "php"],
        "backend" : ["shell", "node.js", "django", "nest.js", "drf restframework", "API", "microservice", "script", "REST API", "CLI", "terminal", "SSH"],
        "languages" : ["python", "javascript", "typescript", "C", "C++", "VBA"],
        "machine  learning" : ["NLP", "Reinforcement learning", "scikit", "Deep learning", "forecasting", "pythorch", "tensorflow", "Pandas", "Numpy"],
        "devops" : ["cloud", "hosting", "docker", "git", "CI/CD"],
        "database" : ["postgres", "SQL", "NoSQL", "MongoDB", "MySQL", "SQLite", "relational database"],
        "data" : ["Power bi", "Tableau"],
        "Other" : ["Agile", "Clean code", "Communication", "Project management", "deadlines", "Debug,selenium", "web automation"]
    },
    
    "education": [
        {
        "degree_name": "Master's degree in Data Science",
        "institution_name": "University of Strasbourg",
        "start_year": "2020",
        "end_year": "2022"
        },
        {
        "degree_name": "Bachelore's degree in Computer Science",
        "institution_name": "University of Strasbourg",
        "start_year": "2017",
        "end_year": "2020"
        }
    ],
}

In [18]:

"skills": "- **front** : html, css, vue.js, react, ux/ui, php\n- **backend** : shell, node.js, django, nest.js, drf restframework, API, microservice, script, REST API, CLI, terminal, SSH\n- **languages** : python, javascript, typescript, C, C++, VBA\n- **machine  learning** : NLP, Reinforcement learning, scikit, Deep learning, forecasting, pythorch, tensorflow, Pandas, Numpy\n- **devops** : cloud, hosting, docker, git, CI/CD\n- **database** : postgres, SQL, NoSQL, MongoDB, MySQL, SQLite, relational database\n- **data** : Power bi, Tableau\n- **Other** : Agile, Clean code, Communication, Project management, deadlines, Debug,selenium, web automation \n",

In [48]:
from jinja2 import Template

In [57]:
def skills_context_to_text(skills_context):
    return '\n'.join(f"- **{k}** : " +  ', '.join(skills_context[k]) for k in skills_context.keys())
    
def work_context_to_text(position_context):
    template_work_experience = """# {{company_name}} - {{position_title}}
**Date : {{start_date}} - {{end_date}}
**Employment Type**: {{employment_type}}
**Responsibilities**:
{{responsibilities}}**Skills and Tools Used**: {{skill_list}}"""


    responsibilities = ''.join(f' - {r}\n' for r in position_context['responsibilities'])
    skills = ', '.join(position_context['skills'])

    context = {
        "company_name" : position_context['company_name'],
        "position_title" : position_context["title"],
        "start_date" : position_context['start_date'],
        "end_date" : position_context['end_date'],
        "employment_type" : position_context['employment_type'],
        "responsibilities" :  responsibilities,
        "skill_list" : skills
    }

    template = Template(template_work_experience)
    output = template.render(context)
    return output

In [83]:
print('\n\n'.join(work_context_to_text(v) for v in data['work_experience'].values()))

# Intrad - Fullstack Developer
**Date : july 2023 - October 2024
**Employment Type**: Full-time
**Responsibilities**:
 - DevOps: Cloud infrastructure selection, deployment scripts (YAML, Bash), database management, domain and hosting management, Git management.
 - E-learning Application Development: Created quiz forms, PDF embedding, certificate generation.
 - CRM Development: API creation for remote control, integration with Discord, Zoom, Vimeo, GSuite, user authentication, email management, customer support, monitoring.
 - Learning Management System (LMS): User resource access control, video tracking, mobile and desktop compatibility.
**Skills and Tools Used**: Microservices, Bash, cloud infrastructure, APIs, Zoom API, GSuite, UX/UI design, database management, python, django, aws, docker, vue.js, node.js, CI/CD

# Freelancing - Fullstack Developer
**Date : september 2020 - October 2024
**Employment Type**: Freelance
**Responsibilities**:
 - ML Engineer: Developper ML models for fin

KeyError: 'company_name'